1. Use whole base model until head, and change the head with a custom head
2. fine tune when you have a large and similar dataset
3. train hole model if you have large and different dataset
4. small and dif --> fine tuning
5. small and similar --> transfer learning

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./resources/cats_and_dogs_filtered.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [16]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile
import os

%matplotlib inline

In [17]:
dataset_path = './resources/cats_and_dogs_filtered.zip'
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
zip_object.extractall("./resources")
zip_object.close()

In [18]:
dataset_path_new = "./resources/cats_and_dogs_filtered"

In [19]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

In [20]:
IMAGE_SIZE = (128, 128, 3)

In [21]:
mobile_net_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SIZE, include_top=False, weights="imagenet")

Change trainable to false to build on it.

In [22]:
mobile_net_model.trainable = False

In [23]:
global_avg_pool = tf.keras.layers.GlobalAveragePooling2D()(mobile_net_model.output)

Prediction layer same size as classes 

In [24]:
prediction_layer = tf.keras.layers.Dense(units=1, activation="sigmoid")(global_avg_pool)

Define model

In [25]:
model = tf.keras.models.Model(inputs=mobile_net_model.input, outputs=prediction_layer)

In [27]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Creating data generators

pre-trained architectures only support certain input sizes

    for example: MobileNet only supports:(96,96), (128,128), (160,160), (192,192), (224,224)

In [28]:
data_gen_train = ImageDataGenerator(rescale=1/255.0)
data_gen_valid = ImageDataGenerator(rescale=1/255.0)

In [29]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [30]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Train Model

In [31]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - ETA: 48s - loss: 0.8734 - accuracy: 0.429 - ETA: 24s - loss: 0.8691 - accuracy: 0.414 - ETA: 16s - loss: 0.8464 - accuracy: 0.447 - ETA: 12s - loss: 0.8480 - accuracy: 0.439 - ETA: 9s - loss: 0.8498 - accuracy: 0.432 - ETA: 8s - loss: 0.8517 - accuracy: 0.43 - ETA: 7s - loss: 0.8429 - accuracy: 0.44 - ETA: 6s - loss: 0.8490 - accuracy: 0.44 - ETA: 5s - loss: 0.8370 - accuracy: 0.45 - ETA: 4s - loss: 0.8322 - accuracy: 0.45 - ETA: 4s - loss: 0.8272 - accuracy: 0.46 - ETA: 3s - loss: 0.8290 - accuracy: 0.46 - ETA: 2s - loss: 0.8276 - accuracy: 0.46 - ETA: 1s - loss: 0.8217 - accuracy: 0.47 - ETA: 0s - loss: 0.8194 - accuracy: 0.47 - 18s 1s/step - loss: 0.8185 - accuracy: 0.4750 - val_loss: 0.7693 - val_accuracy: 0.5030
Epoch 2/5
16/16 [==============================] - ETA: 5s - loss: 0.7547 - accuracy: 0.55 - ETA: 4s - loss: 0.7788 - accuracy: 0.50 - ETA: 4s - loss: 0.7788 - accuracy: 0.51 - ETA: 3s - loss: 0.7836 - accuracy: 0.52 - ETA

In [33]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [35]:
valid_accuracy

0.779

### Fine Tuning

- Do not use fine tuning on the whole network; only a few top layers are enough. In most cases, they are specialized. The goal of the fine tuning is to adopt that specific part of the network for our custom (new) dataset.
- Start  with fine tuning after you have finished with transfer learning step. If we try to perform fine tuning immediately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model.
- On a small dataset fine tuning will cause your model to overfit

In [36]:
mobile_net_model.trainable = True

In [37]:
len(mobile_net_model.layers)

155

In [38]:
fine_tune_at = 100

In [39]:
for layer in mobile_net_model.layers[:fine_tune_at]:
    layer.trainable = False

In [41]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [42]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - ETA: 10s - loss: 0.5120 - accuracy: 0.757 - ETA: 7s - loss: 0.3951 - accuracy: 0.839 - ETA: 6s - loss: 0.3442 - accuracy: 0.86 - ETA: 5s - loss: 0.3034 - accuracy: 0.88 - ETA: 4s - loss: 0.2716 - accuracy: 0.90 - ETA: 4s - loss: 0.2566 - accuracy: 0.90 - ETA: 3s - loss: 0.2418 - accuracy: 0.91 - ETA: 3s - loss: 0.2330 - accuracy: 0.91 - ETA: 2s - loss: 0.2236 - accuracy: 0.92 - ETA: 2s - loss: 0.2164 - accuracy: 0.92 - ETA: 1s - loss: 0.2118 - accuracy: 0.92 - ETA: 1s - loss: 0.2023 - accuracy: 0.92 - ETA: 1s - loss: 0.1927 - accuracy: 0.93 - ETA: 0s - loss: 0.1882 - accuracy: 0.93 - ETA: 0s - loss: 0.1816 - accuracy: 0.93 - 8s 513ms/step - loss: 0.1748 - accuracy: 0.9375 - val_loss: 0.1090 - val_accuracy: 0.9610
Epoch 2/5
16/16 [==============================] - ETA: 5s - loss: 0.0244 - accuracy: 1.00 - ETA: 5s - loss: 0.0304 - accuracy: 0.99 - ETA: 4s - loss: 0.0330 - accuracy: 0.99 - ETA: 4s - loss: 0.0301 - accuracy: 0.99 - ETA: 4s

In [43]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [44]:
print(valid_accuracy)

0.959
